In [1]:
import tensorflow as tf
import numpy as np

2021-11-29 12:09:31.835965: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-29 12:09:31.836007: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
import pickle

In [3]:
with open('../X_smallset.pkl', 'rb') as inp:
    inputs = pickle.load(inp)

In [4]:
single_input = inputs[0]

In [5]:
test_input = [np.array([[0, 1, 1, 0, 0, 1.5],
             [1, 0, 0, 1, 0, 1.2],
             [1, 0, 0, 1, 0, 1.4]]),
             [[[1, [0, 0, 1]], [2, [0, 1, 0]]],
             [[0, [0, 0, 1]]],
             [[0, [0, 1, 0]]]],
             np.array([3, 2, 4, 7])]

In [6]:
features = single_input[0]
adjacency_list = single_input[1]
physics_features = single_input[2]

In [19]:
w_s = tf.Variable(initial_value=tf.initializers.glorot_uniform()(shape=[81, 80], dtype='double'),shape=[81, 80])
w_n = tf.Variable(initial_value=tf.initializers.glorot_uniform()(shape=[103, 80], dtype='double'),shape=[103, 80])

In [55]:
features[0][0:-6]

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 1., 0.])

In [66]:
combination_orders = ([[0, -6], tf.math.add], [[-6, -3], 'position'], [[-3], tf.multiply])

In [73]:
new_features = 80*[None]
for i, adj in enumerate(adjacency_list):
    self_features = tf.reshape(features[i], [1, 81])
    self_conv_features = tf.matmul(self_features, w_s)
    new_features[i] = self_conv_features
    for neighbour in adj:
        neighbour_features = features[neighbour[0]]
        neighbour_bond = neighbour[1]
        new_ordered_features = []
        for i, order in enumerate(combination_orders):
            order_function = order[1]
            indices = order[0]
            
            if i == len(combination_orders) - 1 and len(indices) == 1:
                new_ordered_features[indices[0]:] = order_function(self_features[0][indices[0]:], 
                                                                   neighbour_features[indices[0]:])
            else:
                new_ordered_features[indices[0]:indices[1]] = order_function(self_features[0][indices[0]:indices[1]], 
                                                                   neighbour_features[indices[0]:indices[1]])
            print(new_ordered_features)
    break

[<tf.Tensor: shape=(), dtype=float64, numpy=1.0>, <tf.Tensor: shape=(), dtype=float64, numpy=1.0>, <tf.Tensor: shape=(), dtype=float64, numpy=0.0>, <tf.Tensor: shape=(), dtype=float64, numpy=0.0>, <tf.Tensor: shape=(), dtype=float64, numpy=0.0>, <tf.Tensor: shape=(), dtype=float64, numpy=0.0>, <tf.Tensor: shape=(), dtype=float64, numpy=0.0>, <tf.Tensor: shape=(), dtype=float64, numpy=0.0>, <tf.Tensor: shape=(), dtype=float64, numpy=0.0>, <tf.Tensor: shape=(), dtype=float64, numpy=0.0>, <tf.Tensor: shape=(), dtype=float64, numpy=0.0>, <tf.Tensor: shape=(), dtype=float64, numpy=0.0>, <tf.Tensor: shape=(), dtype=float64, numpy=0.0>, <tf.Tensor: shape=(), dtype=float64, numpy=0.0>, <tf.Tensor: shape=(), dtype=float64, numpy=0.0>, <tf.Tensor: shape=(), dtype=float64, numpy=0.0>, <tf.Tensor: shape=(), dtype=float64, numpy=0.0>, <tf.Tensor: shape=(), dtype=float64, numpy=0.0>, <tf.Tensor: shape=(), dtype=float64, numpy=0.0>, <tf.Tensor: shape=(), dtype=float64, numpy=0.0>, <tf.Tensor: shape=(

TypeError: 'str' object is not callable

In [ ]:
class MultiGraphConvLayer(tf.keras.Model):
    def __init__(self, operations = [[-1], [tf.math.reduce_sum]]):
        super(MultiGraphConvLayer, self).__init__()
    
    def call(self, inputs):
        features = inputs[0]
        adjacency_list = inputs[1]
        physics_features = inputs[2]
        